In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import os

## Import Mobility Data

In [2]:
# keep FIPS as string to preserve leading zeroes
# https://www.google.com/covid19/mobility/

In [3]:
mob = pd.read_csv(r"/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/hia_covid_inputs/2020_US_Region_Mobility_Report.csv", dtype={'census_fips_code': 'str'} )

In [ ]:
sns.displot(mob.date, height=4, aspect=4)

In [5]:
mob['date'] = mob['date'].astype('datetime64')

In [6]:
mob_renam_col = [i.replace('percent_change_from_baseline', 'PCT_CFB') for i in mob.columns]

In [7]:
mob.columns = mob_renam_col

In [8]:
mob_renam_col

['country_region_code',
 'country_region',
 'sub_region_1',
 'sub_region_2',
 'metro_area',
 'iso_3166_2_code',
 'census_fips_code',
 'date',
 'retail_and_recreation_PCT_CFB',
 'grocery_and_pharmacy_PCT_CFB',
 'parks_PCT_CFB',
 'transit_stations_PCT_CFB',
 'workplaces_PCT_CFB',
 'residential_PCT_CFB']

In [9]:
mob.dtypes

country_region_code                      object
country_region                           object
sub_region_1                             object
sub_region_2                             object
metro_area                              float64
iso_3166_2_code                          object
census_fips_code                         object
date                             datetime64[ns]
retail_and_recreation_PCT_CFB           float64
grocery_and_pharmacy_PCT_CFB            float64
parks_PCT_CFB                           float64
transit_stations_PCT_CFB                float64
workplaces_PCT_CFB                      float64
residential_PCT_CFB                     float64
dtype: object

In [10]:
mob.sort_values(by="date", inplace=True)

In [11]:
mob.head(5)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_PCT_CFB,grocery_and_pharmacy_PCT_CFB,parks_PCT_CFB,transit_stations_PCT_CFB,workplaces_PCT_CFB,residential_PCT_CFB
0,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
78743,US,United States,Florida,Escambia County,NaN,NaN,12033,2020-02-15,5.0,0.0,63.0,15.0,3.0,-1.0
552960,US,United States,Tennessee,Claiborne County,NaN,NaN,47025,2020-02-15,-2.0,9.0,NaN,NaN,-5.0,NaN
553394,US,United States,Tennessee,Cocke County,NaN,NaN,47029,2020-02-15,13.0,4.0,-10.0,NaN,4.0,NaN
78459,US,United States,Florida,Duval County,NaN,NaN,12031,2020-02-15,2.0,0.0,10.0,5.0,-1.0,-1.0


## Compute Rolling Average of Mobility Data

In [12]:
list_all_locations = list(mob.census_fips_code.unique())
list_frames = []
for i in list_all_locations:
    frame = mob[mob.census_fips_code == i]
    frame['retail_and_recreation_PCT_CFB_RollingAvg'] = frame.rolling(7, min_periods=7, center=False, on="date")['retail_and_recreation_PCT_CFB'].mean()
    frame['grocery_and_pharmacy_PCT_CFB_RollingAvg'] = frame.rolling(7, min_periods=7, center=False, on="date")['grocery_and_pharmacy_PCT_CFB'].mean()
    frame['parks_PCT_CFB_RollingAvg'] = frame.rolling(7, min_periods=7, center=False, on="date")['parks_PCT_CFB'].mean()
    frame['transit_stations_PCT_CFB_RollingAvg'] = frame.rolling(7, min_periods=7, center=False, on="date")['transit_stations_PCT_CFB'].mean()
    frame['workplaces_PCT_CFB_RollingAvg'] = frame.rolling(7, min_periods=7, center=False, on="date")['workplaces_PCT_CFB'].mean()
    frame['residential_PCT_CFB_RollingAvg'] = frame.rolling(7, min_periods=7, center=False, on="date")['residential_PCT_CFB'].mean()
    list_frames.append(frame)

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [13]:
recombine = pd.concat(list_frames)

In [14]:
mobility_final = recombine

In [15]:
mobility_final[(mobility_final.census_fips_code == "12009") & (mobility_final.date > '2020-10-01')].head(-18)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_PCT_CFB,grocery_and_pharmacy_PCT_CFB,parks_PCT_CFB,transit_stations_PCT_CFB,workplaces_PCT_CFB,residential_PCT_CFB,retail_and_recreation_PCT_CFB_RollingAvg,grocery_and_pharmacy_PCT_CFB_RollingAvg,parks_PCT_CFB_RollingAvg,transit_stations_PCT_CFB_RollingAvg,workplaces_PCT_CFB_RollingAvg,residential_PCT_CFB_RollingAvg
75966,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-02,-18.0,-11.0,-6.0,-16.0,-26.0,6.0,-19.571429,-12.285714,-13.428571,-20.142857,-24.142857,6.142857
75967,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-03,-23.0,-15.0,-34.0,-32.0,-17.0,5.0,-20.000000,-12.857143,-18.857143,-20.285714,-24.428571,6.285714
75968,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-04,-19.0,-13.0,-32.0,-24.0,-19.0,4.0,-20.142857,-13.000000,-22.000000,-19.714286,-24.571429,6.285714
75969,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-05,-22.0,-14.0,-27.0,-14.0,-28.0,7.0,-19.714286,-12.428571,-20.714286,-19.000000,-24.571429,6.142857
75970,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-06,-19.0,-12.0,-10.0,-9.0,-28.0,7.0,-19.571429,-12.428571,-18.428571,-18.000000,-24.714286,6.142857
75971,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-07,-19.0,-11.0,-15.0,-15.0,-27.0,7.0,-19.714286,-12.571429,-18.571429,-17.571429,-24.571429,6.142857
75972,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-08,-20.0,-13.0,-3.0,-8.0,-27.0,7.0,-20.000000,-12.714286,-18.142857,-16.857143,-24.571429,6.142857
75973,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-09,-19.0,-12.0,-14.0,-16.0,-26.0,7.0,-20.142857,-12.857143,-19.285714,-16.857143,-24.571429,6.285714
75974,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-10,-21.0,-14.0,-19.0,-30.0,-16.0,5.0,-19.857143,-12.714286,-17.142857,-16.571429,-24.428571,6.285714
75975,US,United States,Florida,Brevard County,NaN,NaN,12009,2020-10-11,-20.0,-15.0,-18.0,-23.0,-18.0,4.0,-20.000000,-13.000000,-15.142857,-16.428571,-24.285714,6.285714


## Join Full County-Level Dataset

In [16]:
counties = pd.read_csv(r"/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/counties_dataset_full.csv",dtype={'County_FIPS':'str'})

In [17]:
counties.dtypes

Unnamed: 0                 int64
County_FIPS               object
StateCD                   object
Name                      object
HS_DIPLOMA_ONLY_2018     float64
SOME_COLLEGE_2018        float64
BACHELORS_PLUS_2018      float64
Poverty_PCT_2018         float64
UrbanCD                    int64
Median_Income_2018       float64
Population_ACS           float64
Population_Black_ACS     float64
Population_Hisp_ACS      float64
Population_Native_ACS    float64
PCT_Black_ACS            float64
PCT_Hisp_ACS             float64
PCT_Native_ACS           float64
UrbanCD_Description       object
Metro                     object
dtype: object

In [18]:
counties['census_fips_code'] = counties['County_FIPS'].astype('str')

In [19]:
mobility_counties = mobility_final.merge(counties, on = "census_fips_code", how="left")

In [20]:
mobility_counties.dtypes

country_region_code                                 object
country_region                                      object
sub_region_1                                        object
sub_region_2                                        object
metro_area                                         float64
iso_3166_2_code                                     object
census_fips_code                                    object
date                                        datetime64[ns]
retail_and_recreation_PCT_CFB                      float64
grocery_and_pharmacy_PCT_CFB                       float64
parks_PCT_CFB                                      float64
transit_stations_PCT_CFB                           float64
workplaces_PCT_CFB                                 float64
residential_PCT_CFB                                float64
retail_and_recreation_PCT_CFB_RollingAvg           float64
grocery_and_pharmacy_PCT_CFB_RollingAvg            float64
parks_PCT_CFB_RollingAvg                           float

In [21]:
mobility_counties[mobility_counties.census_fips_code == ""]

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_PCT_CFB,grocery_and_pharmacy_PCT_CFB,...,Median_Income_2018,Population_ACS,Population_Black_ACS,Population_Hisp_ACS,Population_Native_ACS,PCT_Black_ACS,PCT_Hisp_ACS,PCT_Native_ACS,UrbanCD_Description,Metro


In [22]:
mobility_counties.head(5)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_PCT_CFB,grocery_and_pharmacy_PCT_CFB,...,Median_Income_2018,Population_ACS,Population_Black_ACS,Population_Hisp_ACS,Population_Native_ACS,PCT_Black_ACS,PCT_Hisp_ACS,PCT_Native_ACS,UrbanCD_Description,Metro
0,US,United States,Florida,Escambia County,NaN,NaN,12033,2020-02-15,5.0,0.0,...,50644.0,309924.0,67398.0,16935.0,1411.0,21.746622,5.464243,0.455273,In small metro area of less than 1 million res...,Smaller_metro
1,US,United States,Florida,Escambia County,NaN,NaN,12033,2020-02-16,1.0,-8.0,...,50644.0,309924.0,67398.0,16935.0,1411.0,21.746622,5.464243,0.455273,In small metro area of less than 1 million res...,Smaller_metro
2,US,United States,Florida,Escambia County,NaN,NaN,12033,2020-02-17,5.0,-2.0,...,50644.0,309924.0,67398.0,16935.0,1411.0,21.746622,5.464243,0.455273,In small metro area of less than 1 million res...,Smaller_metro
3,US,United States,Florida,Escambia County,NaN,NaN,12033,2020-02-18,1.0,-1.0,...,50644.0,309924.0,67398.0,16935.0,1411.0,21.746622,5.464243,0.455273,In small metro area of less than 1 million res...,Smaller_metro
4,US,United States,Florida,Escambia County,NaN,NaN,12033,2020-02-19,7.0,0.0,...,50644.0,309924.0,67398.0,16935.0,1411.0,21.746622,5.464243,0.455273,In small metro area of less than 1 million res...,Smaller_metro


## Bring in the COVID Data to be Joined In 

In [23]:
covid = pd.read_csv(r"/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/covid_dataset_full.csv",dtype={'County_FIPS':'str'})

In [24]:
covid.Date = covid.Date.astype("datetime64")

In [25]:
mobility_counties['Date'] = mobility_counties['date']



In [26]:
covid.shape[0]

1015560

In [27]:
mobility_counties.shape[0]

700522

In [28]:
mobility_covid_urban = covid.merge(mobility_counties, on =['Date','County_FIPS'], how="left")

In [29]:
covid.shape[0]/mobility_covid_urban.shape[0]

0.9824171742108236

In [30]:
# Can't deal with the + signs, so remove these 

large_columns = list(mobility_covid_urban.columns)
large_columns = [x.replace('+','') for x in large_columns]
mobility_covid_urban.columns = large_columns

In [31]:
list(mobility_covid_urban.columns)

['Unnamed: 0_x',
 'Date',
 'Admin2',
 'State',
 'Confirmed',
 'Deaths',
 'Country/Region',
 'Deaths_Previous',
 'Confirmed_Previous',
 'Deaths_New',
 'Deaths_New_7',
 'Deaths_New_14',
 'Deaths_New_21',
 'Deaths_New_28',
 'Confirmed_New',
 'locationcol',
 'Confirmed_New_RollingAvg',
 'Deaths_New_RollingAvg',
 'Deaths_New_7_RollingAvg',
 'Deaths_New_14_RollingAvg',
 'Deaths_New_21_RollingAvg',
 'Deaths_New_28_RollingAvg',
 'County_FIPS',
 'State Code',
 'Region',
 'Division',
 'country_region_code',
 'country_region',
 'sub_region_1',
 'sub_region_2',
 'metro_area',
 'iso_3166_2_code',
 'census_fips_code',
 'date',
 'retail_and_recreation_PCT_CFB',
 'grocery_and_pharmacy_PCT_CFB',
 'parks_PCT_CFB',
 'transit_stations_PCT_CFB',
 'workplaces_PCT_CFB',
 'residential_PCT_CFB',
 'retail_and_recreation_PCT_CFB_RollingAvg',
 'grocery_and_pharmacy_PCT_CFB_RollingAvg',
 'parks_PCT_CFB_RollingAvg',
 'transit_stations_PCT_CFB_RollingAvg',
 'workplaces_PCT_CFB_RollingAvg',
 'residential_PCT_CFB_Roll

In [64]:
columns = [ 'Date',
 'County_FIPS',
 'State Code',
 'Name',
 'retail_and_recreation_PCT_CFB',
 'grocery_and_pharmacy_PCT_CFB',
 'parks_PCT_CFB',
 'transit_stations_PCT_CFB',
 'workplaces_PCT_CFB',
 'residential_PCT_CFB',
 'retail_and_recreation_PCT_CFB_RollingAvg',
 'grocery_and_pharmacy_PCT_CFB_RollingAvg',
 'parks_PCT_CFB_RollingAvg',
 'transit_stations_PCT_CFB_RollingAvg',
 'workplaces_PCT_CFB_RollingAvg',
 'residential_PCT_CFB_RollingAvg',
 'HS_DIPLOMA_ONLY_2018',
 'SOME_COLLEGE_2018',
 'BACHELORS_PLUS_2018',
 'Poverty_PCT_2018',
 'UrbanCD',
 'Median_Income_2018',
 'Population_ACS',
 'Population_Black_ACS',
 'Population_Hisp_ACS',
 'Population_Native_ACS',
 'PCT_Black_ACS',
 'PCT_Hisp_ACS',
 'PCT_Native_ACS',
 'UrbanCD_Description',
 'Metro',
 'State',
 'Confirmed',
 'Deaths',
 'Country/Region',
 'Deaths_Previous',
 'Confirmed_Previous',
 'Deaths_New',
 'Deaths_New_7',
 'Deaths_New_14',
 'Deaths_New_21',
 'Deaths_New_28',
 'Confirmed_New',
 'locationcol',
 'Confirmed_New_RollingAvg',
 'Deaths_New_RollingAvg',
 'Deaths_New_7_RollingAvg',
 'Deaths_New_14_RollingAvg',
 'Deaths_New_21_RollingAvg',
 'Deaths_New_28_RollingAvg',
 ]

In [65]:
combined_dataset_sub = mobility_covid_urban[columns]

In [66]:
combined_dataset_sub

,Date,County_FIPS,State Code,Name,retail_and_recreation_PCT_CFB,grocery_and_pharmacy_PCT_CFB,parks_PCT_CFB,transit_stations_PCT_CFB,workplaces_PCT_CFB,residential_PCT_CFB,...,Deaths_New_21,Deaths_New_28,Confirmed_New,locationcol,Confirmed_New_RollingAvg,Deaths_New_RollingAvg,Deaths_New_7_RollingAvg,Deaths_New_14_RollingAvg,Deaths_New_21_RollingAvg,Deaths_New_28_RollingAvg
0,2020-01-22,01001,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,Autauga|Alabama,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,01001,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,Autauga|Alabama,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,01001,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,Autauga|Alabama,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-25,01001,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,Autauga|Alabama,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-26,01001,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,Autauga|Alabama,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033731,2020-11-22,56045,WY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.0,Weston|Wyoming,4.714286,0.000000,NaN,NaN,NaN,NaN
1033732,2020-11-23,56045,WY,Weston County,NaN,NaN,NaN,NaN,-23.0,NaN,...,NaN,NaN,0.0,Weston|Wyoming,4.857143,0.142857,NaN,NaN,NaN,NaN
1033733,2020-11-24,56045,WY,Weston County,NaN,NaN,NaN,NaN,-21.0,NaN,...,NaN,NaN,11.0,Weston|Wyoming,5.285714,0.142857,NaN,NaN,NaN,NaN
1033734,2020-11-25,56045,WY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,Weston|Wyoming,4.714286,0.142857,NaN,NaN,NaN,NaN


In [67]:
#check to see matching
combined_dataset_sub[['State','State Code']].sample(frac=.00004)


,State,State Code
657287,Ohio,OH
896577,Texas,TX
545175,Nebraska,NE
754885,Puerto Rico,NaN
835969,Texas,TX
419298,Michigan,MI
40965,Arkansas,AR
920444,Utah,UT
505907,Missouri,MO
337991,Kentucky,KY


In [68]:
def create_percapita_measures(dataframe):
    list_col = list(dataframe.columns)
    for i in list(dataframe.columns):
        if "death" in i.lower() or "confirmed" in i.lower() and "previous" not in i.lower() :
            try:
                new_col_name = i+'_PER_100K'
                dataframe[new_col_name] = dataframe[i]/(dataframe['Population_ACS']/100000)
            except:
                print("error occurred for {i}".format(i=i)) 

In [69]:
create_percapita_measures(combined_dataset_sub)

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [70]:
list(combined_dataset_sub.columns)

['Date',
 'County_FIPS',
 'State Code',
 'Name',
 'retail_and_recreation_PCT_CFB',
 'grocery_and_pharmacy_PCT_CFB',
 'parks_PCT_CFB',
 'transit_stations_PCT_CFB',
 'workplaces_PCT_CFB',
 'residential_PCT_CFB',
 'retail_and_recreation_PCT_CFB_RollingAvg',
 'grocery_and_pharmacy_PCT_CFB_RollingAvg',
 'parks_PCT_CFB_RollingAvg',
 'transit_stations_PCT_CFB_RollingAvg',
 'workplaces_PCT_CFB_RollingAvg',
 'residential_PCT_CFB_RollingAvg',
 'HS_DIPLOMA_ONLY_2018',
 'SOME_COLLEGE_2018',
 'BACHELORS_PLUS_2018',
 'Poverty_PCT_2018',
 'UrbanCD',
 'Median_Income_2018',
 'Population_ACS',
 'Population_Black_ACS',
 'Population_Hisp_ACS',
 'Population_Native_ACS',
 'PCT_Black_ACS',
 'PCT_Hisp_ACS',
 'PCT_Native_ACS',
 'UrbanCD_Description',
 'Metro',
 'State',
 'Confirmed',
 'Deaths',
 'Country/Region',
 'Deaths_Previous',
 'Confirmed_Previous',
 'Deaths_New',
 'Deaths_New_7',
 'Deaths_New_14',
 'Deaths_New_21',
 'Deaths_New_28',
 'Confirmed_New',
 'locationcol',
 'Confirmed_New_RollingAvg',
 'Deaths

In [71]:
combined_dataset_sub.dtypes

Date                                 datetime64[ns]
County_FIPS                                  object
State Code                                   object
Name                                         object
retail_and_recreation_PCT_CFB               float64
                                          ...      
Deaths_New_RollingAvg_PER_100K              float64
Deaths_New_7_RollingAvg_PER_100K            float64
Deaths_New_14_RollingAvg_PER_100K           float64
Deaths_New_21_RollingAvg_PER_100K           float64
Deaths_New_28_RollingAvg_PER_100K           float64
Length: 65, dtype: object

In [72]:
combined_dataset_sub.Deaths_New_7_RollingAvg_PER_100K.isna().value_counts()

False    685448
True     348288
Name: Deaths_New_7_RollingAvg_PER_100K, dtype: int64

In [73]:
combined_dataset_sub.Population_ACS.isna().value_counts()

False    697861
True     335875
Name: Population_ACS, dtype: int64

In [74]:
combined_dataset_sub.Deaths_New_7_RollingAvg_PER_100K.isna().value_counts()

False    685448
True     348288
Name: Deaths_New_7_RollingAvg_PER_100K, dtype: int64

In [75]:
combined_dataset_sub.Metro.isna().value_counts()

False    697861
True     335875
Name: Metro, dtype: int64

## Get Regions

In [76]:
regions = pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')

In [77]:
regions= regions.drop(['State'], axis=1)

In [78]:
regions

,State Code,Region,Division
0,AK,West,Pacific
1,AL,South,East South Central
2,AR,South,West South Central
3,AZ,West,Mountain
4,CA,West,Pacific
5,CO,West,Mountain
6,CT,Northeast,New England
7,DC,South,South Atlantic
8,DE,South,South Atlantic
9,FL,South,South Atlantic


In [79]:
nostate = combined_dataset_sub[combined_dataset_sub['State Code'].isna()==True]

In [80]:
combined_dataset_sub_states = combined_dataset_sub[combined_dataset_sub['State Code'].isna()==False]

In [81]:
combined_dataset_sub_regions = combined_dataset_sub_states.merge(regions, on="State Code", how="inner")

In [82]:
combined_dataset_sub_regions.Division.isna().value_counts()

False    1008962
Name: Division, dtype: int64

In [83]:
combined_dataset_sub_regions['Date']

0         2020-01-22
1         2020-01-23
2         2020-01-24
3         2020-01-25
4         2020-01-26
             ...    
1008957   2020-11-22
1008958   2020-11-23
1008959   2020-11-24
1008960   2020-11-25
1008961   2020-11-26
Name: Date, Length: 1008962, dtype: datetime64[ns]

## Export to File

In [84]:

os.chdir(r'/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/')
path = str(os.getcwd())
filename = "hia_covid_combined.csv"
combined_dataset_sub_regions.to_csv(filename)
print("{filename} has been created in {path}".format(filename=filename, path=path))

hia_covid_combined.csv has been created in /Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets


In [85]:

os.chdir(r'/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/')
path = str(os.getcwd())
filename = "hia_covid_combined_sample.csv"
combined_dataset_sub_regions.sample(frac=.10).to_csv(filename)
print("{filename} has been created in {path}".format(filename=filename, path=path))

hia_covid_combined_sample.csv has been created in /Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets


In [86]:
# ratio to check join 
combined_dataset_sub_regions.shape[0]/mobility_counties.shape[0]

1.4403002332546302

In [87]:

os.chdir(r'/Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets/')
path = str(os.getcwd())
filename = "hia_covid_combined_post0601.csv"
combined_dataset_sub_regions[combined_dataset_sub.Date >= '2020-06-01'].to_csv(filename)
print("{filename} has been created in {path}".format(filename=filename, path=path))

/Users/philip.ballentine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


hia_covid_combined_post0601.csv has been created in /Users/philip.ballentine/Documents/hia_covid_repo/hia_covid_data_assets
